# Implementation of Multiple Linear Regression using MPI and OpenMP

## Objectives

* Load the data and perform data pre-processing
* Identify the features, target and split the data into train and test
* Implement multiple Linear Regression by estimating the coefficients on the given data
* Use MPI package to distribute the data and implement `communicator`
* Define functions for each objective and make a script (.py) file to execute using MPI command
* Use OpenMP component to predict the data and calculate the error on the predicted data
* Implement the Linear Regression from `sklearn` and compare the results

### Dataset

The dataset chosen for this mini-project is [Combined Cycle Power Plant](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant). The dataset is made up of 9568 records and 5 columns. Each record contains the values for Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output. The data was collected over a six year period (2006-11).

**Note:** We will be using the mpi4py Python package for MPI based code implementation

In [ ]:
!pip -qq install mpi4py

#### Importing Necessary Packages

In [ ]:
# Importing pandas
import pandas as pd 
# Importing Numpy
import numpy as np 
# Importing MPI from mpi4py package
from mpi4py import MPI 
# Importing sqrt function from the Math
from math import sqrt 
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP 
import time
from sklearn.linear_model import LinearRegression

#### Download the data

#### Load data 


In [ ]:
FILENAME = "/content/PowerPlantData.csv" # File path
def loadData(file):
  df = pd.read_csv(file)
  df.rename(columns = {'AT':'Ambient Temperature', 'V':'Exhaust Vaccum', 'AP':'Ambient Pressure','RH':'Relative Humidity','PE':'Energy Output'}, inplace = True)
  return df

df = loadData(FILENAME)
df.head()

#### Explore data

- checking for the number of rows and columns
- summary of the dataset
- check for the null values 
- check for the duplicate values

In [ ]:
df.head()

In [ ]:
def explore_data(df):
  rows, columns = df.shape
  summary = df.describe()
  nullValues = df.isnull().sum()
  duplicatedValues = df.duplicated().sum()
  return rows, columns, summary, nullValues, duplicatedValues

rows, columns, summary, nullValues, duplicatedValues = explore_data(df)
 
print('\n The shape (rows, columns) of the data set is:', rows, columns)
print('Summary of the dataset:\n', summary)
print('\nNull values in the data set:\n',nullValues)
print('\nNo. of duplicated rows:', duplicatedValues)


#### Handle missing data 

- Replace the null values with the mean/median/mode - fillna()

In [ ]:
from numpy.core.fromnumeric import mean
def handle_data(df):
  for col in df.columns:
    if df[col].isnull().sum() == 0:
      continue
    else:
      df[col] = df[col].fillna(value=df[col].mean())
  return df

df = handle_data(df)
df.isnull().sum()

#### Scale the data 

- standardization of the data  can be performed using the below formula

$ (x - mean(x)) / std(x) $ 

In [ ]:
# Defining a function to standardize the data
def standardize_data(df):
  return (df - df.mean())/df.std()


df_std = standardize_data(df)
df_std.head()

#### Feature selection

- Features: AmbientTemperature, ExhaustVaccum, AmbientPressure, RelativeHumidity 
- Target Variable: EnergyOutput

In [ ]:
def feature_selection(df):
  features = df.drop('Energy Output', axis=1)
  target = df['Energy Output']
  return features, target

X, y = feature_selection(df_std)
X.head()

In [ ]:
y.head()

#### Correlation 

Calculate correlation between the variables

In [ ]:
df.corr()

#### Estimate the coefficients

- Calculate the estimated coefficients using the below formula

$ β = (X^T X)^{-1} X^T y $ 

In [ ]:
# Calculating the coeffients
def estimate_coeff(X,y):
  x_t = X.transpose()
  inv_dot_xt = np.linalg.inv(x_t.dot(X)).dot(x_t)
  coeff = inv_dot_xt.dot(y)
  return coeff

coeff = estimate_coeff(X,y)
coeff

#### Fit the data to estimate the coefficients

- create a dummy column in the features dataframe which is made up of all ones
- convert the features dataframe into numpy array
- call the estimated coefficients function which is defined above

In [ ]:
# defining a fit function
def fit(X, y):
    X1 = np.hstack((np.ones((len(X),1)),X))
    coeff = estimate_coeff(X1,y)
    return coeff, X1

coeff, new_X = fit(X,y)
print('intercept:', coeff[0])
print('coeeficients:', coeff[1:])

In [ ]:
new_X

In [ ]:
new_X.shape

#### Predict the data on estimated coefficients

- Fit the intercept, coefficients values in the below equation

  $y = b_0 + b_1*x + ... + b_i*x_i$

In [ ]:
 # fucntion to predict the values
def predict(x, intercept, coefficients):
    '''
    y = b_0 + b_1*x + ... + b_i*x_i
    '''
    predictions = x.dot(coefficients) + intercept
    return predictions

In [ ]:
y_pred = predict(X, coeff[0], coeff[1:])

In [ ]:
y_pred

#### Root mean squared error

In [ ]:
# Define a function to calculate the error
def rmse(y, y_pred):
  return np.sqrt(np.mean((y - y_pred)**2))

In [ ]:
rms_error = rmse(y, y_pred)
rms_error

#### Split the data into train and test

- Shuffle the data
- Consider 70 % of data as a train set and the rest of the data as a test set

In [ ]:
def train_test_split(x, y, test_size=0.3):
  shuffle_idx = np.random.permutation(x.shape[0])
  X_shuffled, y_shuffled = x[shuffle_idx], y[shuffle_idx]
  count = int((1-test_size)*len(x))
  return X_shuffled[:count,:], y_shuffled[:count], X_shuffled[count:, :], y_shuffled[count:]

In [ ]:
X_copy = X.to_numpy()
y_copy = y.to_numpy()
X_train, y_train, X_test, y_test = train_test_split(X_copy,y_copy, test_size=0.3)

In [ ]:
X_train.shape, y_train.shape

#### Make a script and execute everything in one place using MPI

- create a communicator
- divide the data into slices
- prepare the data in root worker to assign the data to all the workers
-scatter and gather the data
- !mpirun --allow-run-as-root -np 4 python filename.py

In [ ]:
%%writefile mpi_reg.py
# Importing pandas
import pandas as pd 
# Importing Numpy
import numpy as np
# Importing sqrt function from the Math
from math import sqrt 
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP 
import time
from mpi4py import MPI

# Define a function to load the data
def loadData(file):
  df = pd.read_csv(file)
  df.rename(columns = {'AT':'Ambient Temperature', 'V':'Exhaust Vaccum', 'AP':'Ambient Pressure','RH':'Relative Humidity','PE':'Energy Output'}, inplace = True)
  return df

# Explore data to check for no of rows and columns, summary, null and duplicate values
def explore_data(df):
  rows, columns = df.shape
  summary = df.describe()
  nullValues = df.isnull().sum()
  duplicatedValues = df.duplicated().sum()
  return rows, columns, summary, nullValues, duplicatedValues

# Defining a function to standardize the data
def standardize_data(df):
  return (df - df.mean())/df.std()

# function for feature selection
def feature_selection(df):
  features = df.drop('Energy Output', axis=1)
  target = df['Energy Output']
  return features, target

#split data into train and test 
def train_test_split(x, y, test_size=0.3):
  shuffle_idx = np.random.permutation(x.shape[0])
  X_shuffled, y_shuffled = x[shuffle_idx], y[shuffle_idx]
  count = int((1-test_size)*len(x))
  return X_shuffled[:count,:], y_shuffled[:count], X_shuffled[count:, :], y_shuffled[count:]

#Dividing the data into slices for workers
def dividing_data(x_train, y_train, size_of_workers):
    # Size of the slice
    slice_for_each_worker = int(Decimal(x_train.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))      
    print('Slice of data for each worker: {}'.format(slice_for_each_worker))
    data_list = []
    start=0
    end = slice_for_each_worker
    for i in range(size_of_workers):
      data_list.append(np.hstack((x_train[start:end,:], y_train[start:end].reshape(slice_for_each_worker,1))))
      start = end
      end+= slice_for_each_worker
      #data_list.append(np.hstack((x_train[start:end,:], y_train[start:end].reshape(slice_for_each_worker,1))))
    return np.array(data_list)

# Calculating the coefficients
def estimate_coeff(X,y):
  x_t = X.transpose()
  # x_xt = x_t.dot(X)
  # x_xt_inv = np.linalg.inv(x_xt)
  inv_dot_xt = np.linalg.inv(x_t.dot(X)).dot(x_t)
  coeff = inv_dot_xt.dot(y)
  return coeff

# defining a fit function
def fit(X, y):
    # YOUR CODE HERE
    X1 = np.hstack((np.ones((len(X),1)),X))
    coeff = estimate_coeff(X1,y)
    return coeff, X1

def predict(x, intercept, coefficients):
    '''
    y = b_0 + b_1*x + ... + b_i*x_i
    '''
    beta = np.concatenate(([intercept], coefficients))
    predictions = x.dot(beta)
    return predictions

# function for rmse
def rmse(y, y_pred):
  #return ((sum((y-y_pred)**2))/len(y))**0.5
  return np.sqrt(np.mean((y - y_pred)**2))


# Defining a main function 
def main():
    # creating communicator
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank() 
    size = comm.Get_size()

    # Load file
    FILENAME = "/content/PowerPlantData.csv" # File path
    df = loadData(FILENAME)
    print(df.head())

    #Explore data
    rows, columns, summary, nullValues, duplicatedValues = explore_data(df)
    print('\n The shape (rows, columns) of the data set is:', rows, columns)
    print('Summary of the dataset:\n', summary)
    print('\nNull values in the data set:\n',nullValues)
    print('\nNo. of duplicated rows:', duplicatedValues)

    #standardize the data
    df_std = standardize_data(df)
    print('standardized data:\n',df_std.head())

    # feature selection
    X, y = feature_selection(df_std)

    #train test split 
    X_copy = X.to_numpy()
    y_copy = y.to_numpy()
    X_train, y_train, X_test, y_test = train_test_split(X_copy,y_copy, test_size=0.3)
    print('shape of X_train:', X_train.shape)
    print('shape of y_train:', y_train.shape)
    print('shape of X_test:', X_test.shape)
    print('shape of y_test:', y_test.shape)

    
    dataList = None
    # master process
    if rank == 0:
        #dividing the data
        dataList = dividing_data(X_train, y_train, size)
        #print('rank=', rank, 'datalist=', dataList)
    else:
        dataList = None
    sendbuf = np.empty((1674,5))
    comm.Scatter(dataList, sendbuf, root=0)
    print('Rank: ',rank, ', sendbuf received: ',sendbuf[:,-1])
    #print('data list outside rank 0 =', type(dataList))

    rmse_collect = []
    comm.Barrier()
    X_train_subset = sendbuf[:,:-1]
    #print('Rank: ',rank, ', X_train_subset: ',X_train_subset)
    y_train_subset = sendbuf[:,-1]
    print('Rank: ',rank, ', y_train_subset: ',y_train_subset)
    coeff = estimate_coeff(X,y)
    coeff, new_X = fit(X,y)
    print('Rank: ',rank, ', coeff: ',coeff)
    y_pred = predict(new_X, coeff[0], coeff[1:])
    print('Rank: ',rank, ', y_pred: ',y_pred)
    rms_error = rmse(y, y_pred)
    rmse_collect.append(rms_error)
    print('Rank: ',rank, ', rms_error: ',rms_error)
    # worker processes
    

    recvbuf = None
    recvbuf_rmse = None
    if rank == 0:
        # Creating a receiver buffer array
        recvbuf = np.empty(coeff.shape[0] * size)  
        #recvbuf_rmse = np.zeros((4,))

    # Gathering the Information
    comm.Gather(coeff, recvbuf, root = 0)
    #rmse_data = comm.Gather(rmse, recvbuf_rmse, root = 0)
    # Display the result
    if rank == 0:
        print('Rank: ',rank, ', recvbuf received: ',recvbuf)
        print('rmse mean=',sum(rmse_collect)/len(rmse_collect))

    
main()



In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 4 python mpi_reg.py

#### Implement predict using OpenMP

Get the predictions for test data and calculate the test error(RMSE) by implementing the OpenMP (pymp)

* Using the pymp.Parallel implement the predict function (use from above)

* Call the predict function by passing test data as an argument

* calculate the error (RMSE) by comparing the Actual test data and predicted test data

In [ ]:
!pip install pymp-pypi

In [ ]:
X_train.shape

In [ ]:
type(X_train)

In [ ]:
X_test.shape

In [ ]:
pymp.config.nested = True


In [ ]:
incremental_array = pymp.shared.array(X_test.shape)
start = time.perf_counter()
print('start time', start)

with pymp.Parallel(4) as p:
    # Initialize the predicted values array for each thread
    # This will automatically be a shared variable among all threads
    y_pred = np.zeros_like(y_test)

    # Divide the test data into chunks for each thread
    np.copyto(incremental_array, X_test)
    y_pred = predict(incremental_array,coeff[0], coeff[1:])
    
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')
# Calculate the test error (RMSE)
error = np.sqrt(np.mean((y_test - y_pred)**2))
print('Test RMSE: {:.4f}'.format(error))


#### Use Sklearn to compare

Apply the Linear regression on the given data using sklearn package and compare with the above results
* Split the data into train and test
* Fit the train data and predict the test data using `sklearn Linear Regression`
* calculate loss (RMSE) on test data and predictions and compare

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(X_copy,y_copy, test_size=0.3)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred

In [ ]:
rmse_score = rmse(y_test, y_pred)
rmse_score